In [15]:
!pip install adversarial-robustness-toolbox
!pip install pydot
!pip install pydot graphviz
import warnings

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers.legacy import Adam
import numpy as np

#from keras.utils.vis_utils import plot_model
from keras.utils import plot_model

from art.estimators.classification import KerasClassifier
from tensorflow.keras import datasets
from art.utils import load_mnist
warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
                                              0.0/47.0 kB ? eta -:--:--
     ------------------------               30.7/47.0 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 47.0/47.0 kB 471.8 kB/s eta 0:00:00


In [16]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()


model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu", input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))

In [18]:
#print the model summary
#model.summary()

# Plot and save the model architecture
#plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


# Save the model architecture to a file
plot_model(model, to_file='model_architecture.png', show_shapes=True)

# Alternatively, display the model architecture directly
#plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [10]:
from datetime import datetime
Start_time_1 = datetime.now()

In [11]:
model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=0.002), metrics=["accuracy"])

classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

classifier.fit(x_train, y_train, batch_size=64, nb_epochs=50)

In [12]:
End_time_1 = datetime.now()

print('Time taken to train the model: {}'.format(End_time_1 - Start_time_1))

predictions = classifier.predict(x_test)
prediction_image = classifier.predict(x_test)
accuracy = np.sum(np.argmax(prediction_image, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))


Time taken to train the model: 0:47:33.384328
Accuracy on benign test examples: 99.2%


In [13]:
model.save("mnist_trained_runtimeCPU_model.h5")